In [1]:
from os import path
import pandas as pd
from IPython.display import display
import numpy as np 
import sys 
import time 
import zarr

sys.path.append('../src/')

from extract_pixel_data import Extractor

plt.rcParams["font.family"] = ""

In [2]:
file_path = "/Users/apple/Desktop/Akamatsu_Lab/Data/full_movie_output_zarr/cleaned_spots_intensities_c3_all.pkl"
track_df = pd.read_pickle(file_path)

In [3]:
track_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame
0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0
1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0
2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0
3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0
4,183.456690,66.0,1031.0,4.0,2.0,2.0,3.0,0
...,...,...,...,...,...,...,...,...
6231,244.000000,233.0,1931.0,111.0,2.0,2.0,2.0,1
6232,187.333333,239.0,1336.0,110.0,2.0,1.0,2.0,1
6233,234.333333,250.0,1536.0,109.0,2.0,3.0,2.0,1
6234,183.001048,262.0,1312.0,110.0,2.0,3.0,2.0,1


In [4]:
z = zarr.open('/Users/apple/Desktop/Akamatsu_Lab/Data/Zarr_file_new/all_channels_data', mode='r')

In [5]:
z.info

Type,zarr.core.Array
Data type,uint16
Shape,"(130, 3, 116, 2052, 340)"
Chunk shape,"(1, 1, 116, 2052, 340)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,63126086400 (58.8G)
No. bytes stored,13548767504 (12.6G)
Storage ratio,4.7


In [6]:
track_df['frame'].nunique()

2

In [7]:
#Create an extractor object with required parameters to properly run the methods in next steps 
extractor = Extractor(z, dataframe = track_df, radii=[4,2,2], frame_col_name = 'frame', 
                      radi_col_name = ['sigma_z', 'sigma_y', 'sigma_x'], n_jobs =2)

# Extract Information for Channel 3 (Clathrin)

## Extracting voxel sum

In [8]:
# pass the channel for which voxel sum is needed and the coordinates around which voxel sum is supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
voxel_sum_array, _ = extractor.voxel_sum_fixed_bd(center_col_names = ['mu_z', 'mu_y', 'mu_x'], channel = 3)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame is 0
current frame is 1
time taken (seconds) 1.0959067344665527


In [9]:
# pass the channel for which pixel values are needed and the coordinates around which values are supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
mean,maximum,minimum,pixel_values,max_loc = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, channel = 3)

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
time taken (seconds) 1.1340112686157227


In [10]:
max_loc = np.array(max_loc)

In [11]:
track_df['c3_mean_amp'] = mean
track_df['c3_voxel_sum'] = voxel_sum_array
track_df['c3_peak_amp'] = maximum 
track_df['c3_peak_x'] = max_loc[:,2]
track_df['c3_peak_y'] = max_loc[:,1]
track_df['c3_peak_z'] = max_loc[:,0]

In [12]:
track_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame,c3_mean_amp,c3_voxel_sum,c3_peak_amp,c3_peak_x,c3_peak_y,c3_peak_z
0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,134.866667,23859,180,20,1814,2
1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,143.097143,28894,193,25,1805,4
2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,132.306667,25547,187,50,570,4
3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,141.000000,28280,181,134,740,4
4,183.456690,66.0,1031.0,4.0,2.0,2.0,3.0,0,139.165714,27954,187,67,1032,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6231,244.000000,233.0,1931.0,111.0,2.0,2.0,2.0,1,197.528000,34330,257,235,1932,110
6232,187.333333,239.0,1336.0,110.0,2.0,1.0,2.0,1,159.533333,31465,199,241,1337,109
6233,234.333333,250.0,1536.0,109.0,2.0,3.0,2.0,1,227.754286,51741,299,252,1539,107
6234,183.001048,262.0,1312.0,110.0,2.0,3.0,2.0,1,148.777143,30183,179,264,1312,109


# Extract information for Channel 2 

In [13]:
# pass the channel for which pixel values are needed and the coordinates around which values are supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
mean,maximum,minimum,pixel_values,max_loc = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, channel = 2)

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
time taken (seconds) 1.742103099822998


In [14]:
max_loc = np.array(max_loc)

## Finding peak pixel for Channel 2 as not sure about the offset between Channel 3 and Channel 2
**Reason for offset is being shot from different cameras**

In [15]:
track_df['c2_amp'] = mean
track_df['c2_peak'] = maximum
track_df['c2_peak_x'] = max_loc[:,2]
track_df['c2_peak_y'] = max_loc[:,1]
track_df['c2_peak_z'] = max_loc[:,0]

In [16]:
track_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame,c3_mean_amp,c3_voxel_sum,c3_peak_amp,c3_peak_x,c3_peak_y,c3_peak_z,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z
0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,134.866667,23859,180,20,1814,2,112.226667,131,21,1813,5
1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,143.097143,28894,193,25,1805,4,114.851429,142,25,1806,5
2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,132.306667,25547,187,50,570,4,106.986667,123,50,570,5
3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,141.000000,28280,181,134,740,4,120.811429,148,135,739,5
4,183.456690,66.0,1031.0,4.0,2.0,2.0,3.0,0,139.165714,27954,187,67,1032,5,117.045714,145,66,1033,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6231,244.000000,233.0,1931.0,111.0,2.0,2.0,2.0,1,197.528000,34330,257,235,1932,110,118.512000,146,232,1929,110
6232,187.333333,239.0,1336.0,110.0,2.0,1.0,2.0,1,159.533333,31465,199,241,1337,109,124.040000,151,238,1337,112
6233,234.333333,250.0,1536.0,109.0,2.0,3.0,2.0,1,227.754286,51741,299,252,1539,107,144.022857,203,248,1539,107
6234,183.001048,262.0,1312.0,110.0,2.0,3.0,2.0,1,148.777143,30183,179,264,1312,109,120.885714,156,263,1312,109


## Finding mean value around the peak pixel value for channel 2 

In [17]:
start_time = time.time()
col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x']
peak_mean,maxima,_,_,_ = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, 
                                                     channel = 2)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
time taken (seconds) 1.2303516864776611


In [18]:
track_df['c2_peak_mean'] = peak_mean

In [19]:
track_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame,c3_mean_amp,c3_voxel_sum,c3_peak_amp,c3_peak_x,c3_peak_y,c3_peak_z,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z,c2_peak_mean
0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,134.866667,23859,180,20,1814,2,112.226667,131,21,1813,5,119.653333
1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,143.097143,28894,193,25,1805,4,114.851429,142,25,1806,5,115.931429
2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,132.306667,25547,187,50,570,4,106.986667,123,50,570,5,108.906667
3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,141.000000,28280,181,134,740,4,120.811429,148,135,739,5,119.777143
4,183.456690,66.0,1031.0,4.0,2.0,2.0,3.0,0,139.165714,27954,187,67,1032,5,117.045714,145,66,1033,7,119.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6231,244.000000,233.0,1931.0,111.0,2.0,2.0,2.0,1,197.528000,34330,257,235,1932,110,118.512000,146,232,1929,110,125.424000
6232,187.333333,239.0,1336.0,110.0,2.0,1.0,2.0,1,159.533333,31465,199,241,1337,109,124.040000,151,238,1337,112,119.360000
6233,234.333333,250.0,1536.0,109.0,2.0,3.0,2.0,1,227.754286,51741,299,252,1539,107,144.022857,203,248,1539,107,177.102857
6234,183.001048,262.0,1312.0,110.0,2.0,3.0,2.0,1,148.777143,30183,179,264,1312,109,120.885714,156,263,1312,109,123.177143


## Finding voxel sum around peak for channel 2 

In [20]:
# pass the channel for which voxel sum is needed and the coordinates around which voxel sum is supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
voxel_sum_array, _ = extractor.voxel_sum_fixed_bd(center_col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'],
                                       channel = 2)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame is 0
current frame is 1
time taken (seconds) 0.7251451015472412


In [21]:
#calculated around the peak value coordinates
track_df['c2_voxel_sum'] = voxel_sum_array

In [22]:
track_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame,c3_mean_amp,c3_voxel_sum,...,c3_peak_x,c3_peak_y,c3_peak_z,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z,c2_peak_mean,c2_voxel_sum
0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,134.866667,23859,...,20,1814,2,112.226667,131,21,1813,5,119.653333,26661
1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,143.097143,28894,...,25,1805,4,114.851429,142,25,1806,5,115.931429,26147
2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,132.306667,25547,...,50,570,4,106.986667,123,50,570,5,108.906667,24234
3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,141.000000,28280,...,134,740,4,120.811429,148,135,739,5,119.777143,26840
4,183.456690,66.0,1031.0,4.0,2.0,2.0,3.0,0,139.165714,27954,...,67,1032,5,117.045714,145,66,1033,7,119.800000,26872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6231,244.000000,233.0,1931.0,111.0,2.0,2.0,2.0,1,197.528000,34330,...,235,1932,110,118.512000,146,232,1929,110,125.424000,28319
6232,187.333333,239.0,1336.0,110.0,2.0,1.0,2.0,1,159.533333,31465,...,241,1337,109,124.040000,151,238,1337,112,119.360000,23988
6233,234.333333,250.0,1536.0,109.0,2.0,3.0,2.0,1,227.754286,51741,...,252,1539,107,144.022857,203,248,1539,107,177.102857,41434
6234,183.001048,262.0,1312.0,110.0,2.0,3.0,2.0,1,148.777143,30183,...,264,1312,109,120.885714,156,263,1312,109,123.177143,27423


### Gaussian Fitting for Channel 2 around peak values 

In [23]:
#Calculating the gaussian fitting estimates around peak coords for channel 2 
#expected sigma value needed for gaussian fitting 
#set all frames to False for processing limited frames 
#max_frames determines the frames to be processed if all_frames is false
start_time = time.time()
channel2_gaussians_df = extractor.run_parallel_frame_processing(expected_sigma = [4,2,2], 
                                        center_col_name = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'], 
                                       dist_between_spots = 10 , channel = 2,  max_frames =  2, all_frames = False)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

Processing frames: 100%|██████████████████████████| 2/2 [00:54<00:00, 27.25s/it]


10%(310 of 3094)
20%(619 of 3094)
30%(929 of 3094)
40%(1238 of 3094)
50%(1547 of 3094)
60%(1857 of 3094)
70%(2166 of 3094)
80%(2476 of 3094)
90%(2785 of 3094)
100%(3094 of 3094)
(3094, 7)
the gaussian did not fit
the number of times the gaussian fitting worked was 3093 and the number of times the gaussian did not fit was 1
10%(315 of 3142)
20%(629 of 3142)
30%(943 of 3142)
40%(1257 of 3142)
50%(1571 of 3142)
60%(1886 of 3142)
70%(2200 of 3142)
80%(2514 of 3142)
90%(2828 of 3142)
100%(3142 of 3142)
(3142, 7)
the gaussian did not fit
the gaussian did not fit
the number of times the gaussian fitting worked was 3140 and the number of times the gaussian did not fit was 2
time taken (seconds) 54.78116202354431


In [24]:
track_df['c2_gaussian_amp'] = channel2_gaussians_df['amplitude']
track_df['c2_mu_x'] = channel2_gaussians_df['mu_x']
track_df['c2_mu_y'] = channel2_gaussians_df['mu_y']
track_df['c2_mu_z'] = channel2_gaussians_df['mu_z']
track_df['c2_sigma_x'] = channel2_gaussians_df['sigma_x']
track_df['c2_sigma_y'] = channel2_gaussians_df['sigma_y']
track_df['c2_sigma_z'] = channel2_gaussians_df['sigma_z']

In [25]:
track_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame,c3_mean_amp,c3_voxel_sum,...,c2_peak_z,c2_peak_mean,c2_voxel_sum,c2_gaussian_amp,c2_mu_x,c2_mu_y,c2_mu_z,c2_sigma_x,c2_sigma_y,c2_sigma_z
0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,134.866667,23859,...,5,119.653333,26661,131.010603,22.0,1817.0,5.0,7.0,13.0,7.0
1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,143.097143,28894,...,5,115.931429,26147,138.621950,25.0,1807.0,3.0,3.0,3.0,5.0
2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,132.306667,25547,...,5,108.906667,24234,118.000000,49.0,569.0,4.0,4.0,2.0,6.0
3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,141.000000,28280,...,5,119.777143,26840,144.638973,134.0,738.0,4.0,3.0,3.0,5.0
4,183.456690,66.0,1031.0,4.0,2.0,2.0,3.0,0,139.165714,27954,...,7,119.800000,26872,141.265776,65.0,1032.0,6.0,2.0,3.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6231,244.000000,233.0,1931.0,111.0,2.0,2.0,2.0,1,197.528000,34330,...,110,125.424000,28319,142.107996,231.0,1926.0,107.0,3.0,7.0,6.0
6232,187.333333,239.0,1336.0,110.0,2.0,1.0,2.0,1,159.533333,31465,...,112,119.360000,23988,146.000000,238.0,1336.0,111.0,3.0,2.0,4.0
6233,234.333333,250.0,1536.0,109.0,2.0,3.0,2.0,1,227.754286,51741,...,107,177.102857,41434,204.247782,247.0,1538.0,103.0,3.0,2.0,7.0
6234,183.001048,262.0,1312.0,110.0,2.0,3.0,2.0,1,148.777143,30183,...,109,123.177143,27423,151.000000,265.0,1311.0,108.0,6.0,2.0,4.0


# Extract information for channel 1

In [26]:
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
radi_list = ['sigma_z', 'sigma_y', 'sigma_x']
mean,maximum,minimum,pixel_values,max_loc = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, channel = 1)

current frame number is 0
current frame number is 1


In [27]:
max_loc = np.array(max_loc)

In [28]:
track_df['c1_amp'] = mean
track_df['c1_peak'] = maximum
track_df['c1_peak_x'] = max_loc[:,2]
track_df['c1_peak_y'] = max_loc[:,1]
track_df['c1_peak_z'] = max_loc[:,0]

In [29]:
voxel_sum_array, _ = extractor.voxel_sum_fixed_bd(center_col_names = ['mu_z', 'mu_y', 'mu_x'], channel = 1)


current frame is 0
current frame is 1


In [30]:
#calculated around the peak value coordinates
track_df['c1_voxel_sum'] = voxel_sum_array

In [31]:
track_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame,c3_mean_amp,c3_voxel_sum,...,c2_mu_z,c2_sigma_x,c2_sigma_y,c2_sigma_z,c1_amp,c1_peak,c1_peak_x,c1_peak_y,c1_peak_z,c1_voxel_sum
0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,134.866667,23859,...,5.0,7.0,13.0,7.0,134.720000,156,19,1813,5,24280
1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,143.097143,28894,...,3.0,3.0,3.0,5.0,139.565714,167,24,1806,5,28224
2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,132.306667,25547,...,4.0,4.0,2.0,6.0,118.173333,138,50,568,4,23594
3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,141.000000,28280,...,4.0,3.0,3.0,5.0,164.662857,217,132,738,7,33761
4,183.456690,66.0,1031.0,4.0,2.0,2.0,3.0,0,139.165714,27954,...,6.0,2.0,3.0,5.0,141.022857,178,65,1033,6,28395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6231,244.000000,233.0,1931.0,111.0,2.0,2.0,2.0,1,197.528000,34330,...,107.0,3.0,7.0,6.0,343.680000,530,235,1931,109,63662
6232,187.333333,239.0,1336.0,110.0,2.0,1.0,2.0,1,159.533333,31465,...,111.0,3.0,2.0,4.0,327.906667,482,241,1335,108,62876
6233,234.333333,250.0,1536.0,109.0,2.0,3.0,2.0,1,227.754286,51741,...,103.0,3.0,2.0,7.0,544.302857,760,250,1537,107,127329
6234,183.001048,262.0,1312.0,110.0,2.0,3.0,2.0,1,148.777143,30183,...,108.0,6.0,2.0,4.0,215.634286,278,260,1311,108,43403


In [32]:
track_df.shape

(6236, 34)

In [33]:
file_path = "/Users/apple/Desktop/Akamatsu_Lab/Data/full_movie_output_zarr/track_df_cleaned_final.pkl"
track_df.to_pickle(file_path)

### Perform Guassian Fitting for channel 1

In [34]:
start_time = time.time()
channel1_gaussians_df = extractor.run_parallel_frame_processing(expected_sigma = [4,2,2], 
                                        center_col_name = ['c1_peak_z', 'c1_peak_y', 'c1_peak_x'], 
                                       dist_between_spots = 10 , channel = 1,  max_frames =  2, all_frames = False)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

Processing frames: 100%|██████████████████████████| 2/2 [00:56<00:00, 28.12s/it]


10%(310 of 3094)
20%(619 of 3094)
30%(929 of 3094)
40%(1238 of 3094)
50%(1547 of 3094)
60%(1857 of 3094)
70%(2166 of 3094)
80%(2476 of 3094)
90%(2785 of 3094)
100%(3094 of 3094)
(3094, 7)
the number of times the gaussian fitting worked was 3094 and the number of times the gaussian did not fit was 0
10%(315 of 3142)
20%(629 of 3142)
30%(943 of 3142)
40%(1257 of 3142)
50%(1571 of 3142)
60%(1886 of 3142)
70%(2200 of 3142)
80%(2514 of 3142)
90%(2828 of 3142)
100%(3142 of 3142)
(3142, 7)
the number of times the gaussian fitting worked was 3142 and the number of times the gaussian did not fit was 0
time taken (seconds) 56.530609369277954


In [35]:
channel1_gaussians_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame
0,153.208748,19.0,1810.0,5.0,2.0,10.0,5.0,0
1,163.445330,24.0,1804.0,9.0,4.0,2.0,12.0,0
2,136.305986,49.0,567.0,4.0,2.0,2.0,3.0,0
3,215.333333,135.0,737.0,8.0,9.0,3.0,6.0,0
4,174.794048,65.0,1032.0,10.0,3.0,2.0,12.0,0
...,...,...,...,...,...,...,...,...
6231,531.486064,235.0,1932.0,107.0,2.0,5.0,5.0,1
6232,478.566128,240.0,1335.0,106.0,2.0,2.0,4.0,1
6233,760.441756,249.0,1536.0,104.0,3.0,3.0,6.0,1
6234,276.430489,259.0,1310.0,107.0,3.0,3.0,4.0,1


In [36]:
track_df['c1_gaussian_amp'] = channel1_gaussians_df['amplitude']
track_df['c1_mu_x'] = channel1_gaussians_df['mu_x']
track_df['c1_mu_y'] = channel1_gaussians_df['mu_y']
track_df['c1_mu_z'] = channel1_gaussians_df['mu_z']
track_df['c1_sigma_x'] = channel1_gaussians_df['sigma_x']
track_df['c1_sigma_y'] = channel1_gaussians_df['sigma_y']
track_df['c1_sigma_z'] = channel1_gaussians_df['sigma_z']

In [37]:
track_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame,c3_mean_amp,c3_voxel_sum,...,c1_peak_y,c1_peak_z,c1_voxel_sum,c1_gaussian_amp,c1_mu_x,c1_mu_y,c1_mu_z,c1_sigma_x,c1_sigma_y,c1_sigma_z
0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,134.866667,23859,...,1813,5,24280,153.208748,19.0,1810.0,5.0,2.0,10.0,5.0
1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,143.097143,28894,...,1806,5,28224,163.445330,24.0,1804.0,9.0,4.0,2.0,12.0
2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,132.306667,25547,...,568,4,23594,136.305986,49.0,567.0,4.0,2.0,2.0,3.0
3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,141.000000,28280,...,738,7,33761,215.333333,135.0,737.0,8.0,9.0,3.0,6.0
4,183.456690,66.0,1031.0,4.0,2.0,2.0,3.0,0,139.165714,27954,...,1033,6,28395,174.794048,65.0,1032.0,10.0,3.0,2.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6231,244.000000,233.0,1931.0,111.0,2.0,2.0,2.0,1,197.528000,34330,...,1931,109,63662,531.486064,235.0,1932.0,107.0,2.0,5.0,5.0
6232,187.333333,239.0,1336.0,110.0,2.0,1.0,2.0,1,159.533333,31465,...,1335,108,62876,478.566128,240.0,1335.0,106.0,2.0,2.0,4.0
6233,234.333333,250.0,1536.0,109.0,2.0,3.0,2.0,1,227.754286,51741,...,1537,107,127329,760.441756,249.0,1536.0,104.0,3.0,3.0,6.0
6234,183.001048,262.0,1312.0,110.0,2.0,3.0,2.0,1,148.777143,30183,...,1311,108,43403,276.430489,259.0,1310.0,107.0,3.0,3.0,4.0


# To test the following functions change the markdown files to code files 
1. Voxel sum using the variable sigma/radi values directly inferred from the dataset 
2. Extracting pixel values mean,max etc using fixed radi
3. Performing Gaussian Fitting on peak coords for single frame

### Function 1 (Voxel Sum)
start_time = time.time()
voxel_sum_array_variable, _ = extractor.voxel_sum_variable_bd(dataframe = track_df,center_col_names = ['mu_z', 'mu_y', 'mu_x'],
                                                  radi_col_name = ['sigma_z','sigma_y','sigma_x'], frame_col_name = 'frame', channel = 3)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

### Function 2 (Extracting Pixel Values)
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
radii = [4, 2, 2]
mean_v,maximum_v,minimum_v,pixel_values_v,max_loc_v = extractor.extract_pixels_data_fixed_bd(center_col_names = col_names,
                dataframe = track_df, radii = radii, frame_col_name = 'frame', channel = 3)

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

### Function 3 (Gaussian fitting on one frame for one channel)
df = extractor.gaussian_fitting_single_frame(expected_sigma = [4,2,2], 
                              center_col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'],
                                      frame = 0, channel = 2, dist_between_spots = 10)